# Simulating the X-ray luminosity function (XLF) of an X-ray binary (XRB) population

In the previous labs, you explored populations of double compact object mergers, supernovae, and gamma-ray bursts. These phenomena are instantaneous transient events: they occur on timescales much shorter than the typical lifetime of a binary system. Within the POSYDON framework, such events can be modeled efficiently using an initial–final interpolation scheme, which maps the initial binary properties directly to their final, post-event outcome.

X-ray binaries, however, are fundamentally different. They represent evolutionary phases of binary systems that persist for a non-negligible fraction of the system’s lifetime. During these phases, the properties of the binaries—such as component masses, orbital periods, and X-ray luminosities—evolve continuously with time. Because of this temporal evolution, an initial–final interpolation approach is not sufficient to capture their behavior.

### Modeling strategies for X-ray binary populations

There are two principal approaches to model populations of systems like X-ray binaries:
1.  Snapshot method
-   Evolve each binary up to a pre-specified age.
-   Record its properties at that moment.
-   If the binary is in an X-ray phase at that time, include it in the population statistics.
2.  Full evolutionary history method (Lab2)
-   Evolve each binary from zero-age to the end of its life.
-   Identify all intervals during which the system qualifies as an X-ray binary.
-   Record the evolving properties of the system across these intervals, weighting them by the time spent in each state.

The "snapshot" method is conceptually straightforward but computationally inefficient: a binary may have been an X-ray source at earlier or later times, and its observable properties evolve during the X-ray phase, which is not fully captured by a single snapshot. This "full evolutionary history" method is more computationally efficient and captures the temporal evolution of the population in greater detail. It is however conceptualy more complex to implement.

![XRB schematic](xrb_schematic.png)
<details>
<summary>Code to produce this diagram</summary>

~~~python

import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(11, 3))

# Timeline
ax.hlines(1, 0, 11, color="black")
ax.set_ylim(0.5, 1.6)
ax.set_xlim(0, 11)

# Key events
events = [0, 2, 8, 10]
labels = ["ZAMS", "1st Supernova", "2nd Supernova", "DCO formation"]
colors = ["black", "red", "red", "orange"]

for x, label, c in zip(events, labels, colors):
    ax.plot(x, 1, "o", color=c)
    ax.text(x, 1.12, label, ha="center", color=c, fontsize=10)

# X-ray binary phase between the two SNe
ax.hlines(0.9, 3, 7, color="blue", linewidth=6, alpha=0.3)
ax.text(5, 0.72, "X-ray Binary Phase", ha="center", color="blue")

# Snapshot method arrow (single age cut)
ax.annotate("Snapshot method\n(one age)", xy=(6, 1.05), xytext=(6, 1.38),
            arrowprops=dict(arrowstyle="->", color="darkgreen"),
            ha="center", color="darkgreen", fontsize=9)

# Full history method bracket
ax.annotate("", xy=(3, 1.28), xytext=(7, 1.28),
            arrowprops=dict(arrowstyle="|-|", color="purple", linewidth=2))
ax.text(5, 1.35, "Full history method", ha="center", color="purple", fontsize=9)

# Formatting
ax.axis("off")
ax.set_title("From ZAMS to Double Compact Object Formation", fontsize=14)

plt.savefig("xrb_schematic.png", dpi=150, bbox_inches="tight")

plt.show()

~~~

</details>



In this first lab, we will adopt the snapshot method: evolving binaries to a fixed age and recording their properties if they are X-ray binaries at that time. This will introduce the concepts and workflow for simulating X-ray binary populations. In the following lab, we will implement the full evolutionary history method, which provides a more complete and efficient treatment of these systems.



**By the end of this exercise, you should be able to:**
-   Set up and run a POSYDON population synthesis calculation appropriate for the modelling of high-mass X-ray binaries.
-   Load and inspect simulated populations, extracting key information such as the number of systems, simulated stellar mass, and their formation pathways.
-   Identify and select X-ray binaries within the broader population at a fixed age, using criteria based on stellar states and accretion properties.
-   Compute and analyze the X-ray luminosity function (XLF) by combining the luminosities of the simulated XRBs into a population distribution.
-   Visualize and interpret the results, linking simulation outputs to astrophysical observables, and discussing how population synthesis can be used to test models of binary evolution against X-ray observations of galaxies.



## 1.   Setting up the population run

First, let's load some required packages.


In [ ]:
#import posydon
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams["text.usetex"] = False
mpl.rcParams["font.family"] = "DejaVu Serif"

from posydon.config import PATH_TO_POSYDON, PATH_TO_POSYDON_DATA

# importing path to our lab data
data_path = os.path.join(os.path.dirname(PATH_TO_POSYDON_DATA), "2025_school_data/populations/XRB_10_pops")

then, copy default population parameters file to current directory



In [ ]:
path_to_params = os.path.join(PATH_TO_POSYDON, "posydon/popsyn/population_params_default.ini")
shutil.copyfile(path_to_params, './population_params.ini')

We need to open the population_params.ini file that we just copied and edit it to adjust the parameters we need for our simulations. As in the previous lab, we will explore high-mass X-ray binaries, in start-forming galaxies. Often, star-formation rates in galaxies are measured assuming a constant star-foramtion rate over the last 100Myr. This is excactly what we will assume hear as well. What are the changes that we would need to make in population_params.ini file?

<details>
<summary>Click to reveal code</summary>

~~~python
star_formation = 'constant'
# 'constant', 'burst', 'custom_linear', 'custom_log10',
# 'custom_linear_histogram', 'custom_log10_histogram'
max_simulation_time = 1e8
# float (0, inf)
~~~

</details>

Let's also define that we want to run 100 binaries at two different metallicities, Solar and 10% Solar.

<details>
<summary>Click to reveal code</summary>

~~~python
  number_of_binaries = 100
    # int (0, inf)
  metallicity = [1., 0.1]
    # in units of solar metallicity: list of float
    # e.g. [2., 1., 0.45, 0.2, 0.1, 0.01, 0.001, 0.0001]
~~~

</details>

Now it is time to run our test population. We will use as before the `PopulationRunner` class to set up the run, by reading the parameters from the file we just copied and edited to our current directory, and then run it with the `evolve` method. This might take a few minutes...


In [ ]:
from posydon.popsyn.synthetic_population import PopulationRunner
poprun = PopulationRunner('./population_params.ini', verbose=True)
poprun.evolve()

We now need to load back the file of the population we run. Let's load the Solar metalicity population using the `Population` class.


In [ ]:
from posydon.popsyn.synthetic_population import Population

pop_path = os.path.join(data_path, '1e+00_Zsun_population.h5')
pop = Population(pop_path)

There, we can check for things like the simulated mass, the number of systems, or look in a bit more detail as specific binary.


In [ ]:
print(pop.mass_per_metallicity)
# you can also access the total number of systems in the file with
print(pop.number_of_systems)
# select only binary_index 5
print(pop.history[5])
# check the history lengths of binaries
print(pop.history_lengths)
# Calculate and inspect the different formation channels
pop.calculate_formation_channels(mt_history=True)
# the formation channels are loaded in with pop.formation_channels
pop.formation_channels

Unfortunately, with a 10 binary population, we cannot do much. To get a better feeling of what the population synthesis can do, we will use a pre-calculated population with 100,000 binaries at Z=0.0142 (solar metallicity). The file is called `1e+00_Zsun_population.h5` and it is located in the `data` folder of posydon. You can load it directly from there using the `Population` class.

In [ ]:
data_path = os.path.join(os.path.dirname(PATH_TO_POSYDON_DATA), "2025_school_data/populations/XRB_100K_pops")
pop_path = os.path.join(data_path, '1e+00_Zsun_population.h5')

pop = Population(pop_path)

<div class='alert alert-success'>

### Excercise 1
Check the total number of binaries in this population, the simulated mass, how many unique formation pathways the binaries of this population have followed, and how many binaries have gone through each channel. 

Since in this lab, we only care about the properties of the binary at the end of our simulations, i.e. at 100Myr, we do not need to deal in most cases with the `history` but we can look at the `oneline` instead, which contains the initial and final properties of the binary. Thus, it would be handy to print a list of all the `keys` of `oneline`.
    
</div>

In [ ]:
#Write here your code for Excercise 1

<div class="alert alert-warning" style="margin-top: 20px">
<details>
    
<b><summary>Solution (click to reveal):</summary></b>

~~~python
print("The simulated mass  and numbe of binaries per metallicity is:")
print(pop.mass_per_metallicity)
print('')

# Calculate and inspect the different formation channels
pop.calculate_formation_channels(mt_history=True)
print('')

# the formation channels are loaded in with pop.formation_channels.
print("The number of binaries following each formation channels are:")
print(pop.formation_channels.value_counts())
print('')

#The keys of oneline are:
print(", ".join(list(pop.oneline[0].keys())))
print('')
~~~

</details>

## 2.   Selecting the XRB population

Out of the entire population that we simulated, only a small fraction are potentially XRBs. So we do not need to be carrying everything around. To do that, we are gonna try to follow as closely as possible the tutorial on ["Transient populations"](https://posydon.org/POSYDON/latest/tutorials-examples/population-synthesis/bbh_analysis.html). You can consult that for inspiration. However, Since here we are dealing with XRBs and not transients, some of the steps will have to be adapted.

<div class='alert alert-success'>
    
### Excercise 2
First, let's filter our populations to only select potential XRBs. For a binary to be an XRB, one of the two components must be a neutron star or a black hole, while the other must be a normal, non-compact star. Of course, some binaries get disrupted during the supernova. We need to get rid of those as well. The goal to save the indices of all binaries that may be XRBs into a list that we will name `selected_indices`.

</div>
    
<div class="alert alert-warning" style="margin-top: 20px">
<details>
    
<b><summary>Hint (click to reveal):</summary></b>

From the states of compact object companions, apart from `NS` and `BH`  we also want to exclude the states `WD` and `massless remnant`.

Following the tutorial logic, we need to create a temporary dataframe from the `oneline` that includes the fields `S1_state_f`,  `S1_state_f`, and `state_f`. That way, it will be easier to buid a mask that implements our logic. 


</details>

In [ ]:
# write your code for Excercise 2

<div class="alert alert-warning" style="margin-top: 20px">
<details>
    
<b><summary>Solution (click to reveal):</summary></b>

~~~python

tmp_data = pop.oneline.select(columns=['S1_state_f', 'S2_state_f'])
S1_state = tmp_data['S1_state_f']
S2_state = tmp_data['S2_state_f']

# get the indices of all systems
indices = tmp_data.index

compact = {'BH', 'NS'}
exclude = {'BH', 'NS', 'massless_remnant', 'WD'}

mask = (S1_state.isin(compact) & ~S2_state.isin(exclude)) | (S2_state.isin(compact) & ~S1_state.isin(exclude))

# get the indices that satisfy all the conditions
selected_indices = indices[mask].to_list()

print(selected_indices)
pop.oneline[selected_indices]

~~~

</details>

Before we continue, let's save this filtered population into a new file (e.g. `XRBs.h5`), and then reload it into a new population that we will call `XRB_pop`.


In [ ]:
# set overwrite to False to add to the file
pop.export_selection(selected_indices, 'XRBs.h5', append=False, overwrite=True)

XRB_pop = Population('XRBs.h5')
print(XRB_pop.number_of_systems)

#Note that the simulated mass is the same as what was reported before the filtering. This information is retained.
print(XRB_pop.mass_per_metallicity)

The next step in the ["Transient populations"](https://posydon.org/POSYDON/latest/tutorials-examples/population-synthesis/bbh_analysis.html) tutorial is to build a selection function. There, we basically need to constract a pandas dataframe which contains only the information that we need for our further analysis. We will build this step by step, as in the tutorial.

For the later calculations we are gonna do, it is convenient to switch from an S1 / S2 notation to donnor / accretor notation. To do that, we will need in the selection funtion to loop over the systems and check which one is the compact object (BH or NS) and assign that to be the accretor, while the other star will be the donor. In contrast to the the tutorial for "transients" where we want to be searching the history, here we only care about the final state, so the oneline is sufficient.



In [ ]:
def XRB_selection_function(history_chunk, oneline_chunk, formation_channels_chunk=None):
    '''A XRB selection function to create a population of XRBs, where we store only the necessary information.'''

    indices = oneline_chunk.index.to_numpy()
    df_XRBs = pd.DataFrame(index = indices)


    df_XRBs['time'] = oneline_chunk['time_f'] * 1e-6 #Myr
    df_XRBs['metallicity'] = oneline_chunk['metallicity'] #This is not really necessary in our case, since we have one file per metallicity, but it is required by the fanctionality of the "transient population we will use"

  
    # Compact object types
    compact_types = {'BH', 'NS'}

    # Prepare new columns
    donor_mass = []
    accretor_mass = []
    donor_type = []
    accretor_type = []


    # Loop over each system
    for s1, s2, m1, m2 in zip(oneline_chunk['S1_state_f'], oneline_chunk['S2_state_f'], oneline_chunk['S1_mass_f'], oneline_chunk['S2_mass_f']):
        if s1 in compact_types and s2 not in compact_types:
            accretor_mass.append(m1)
            accretor_type.append(s1)
            donor_mass.append(m2)
            donor_type.append(s2)

        elif s2 in compact_types and s1 not in compact_types:
            accretor_mass.append(m2)
            accretor_type.append(s2)
            donor_mass.append(m1)
            donor_type.append(s1)

        else:
            # If neither or both are compact, fill with NaN or original values
            accretor_mass.append(float('nan'))
            accretor_type.append(None)
            donor_mass.append(float('nan'))
            donor_type.append(None)

    # Add new columns to DataFrame
    df_XRBs['donor_mass'] = donor_mass
    df_XRBs['accretor_mass'] = accretor_mass
    df_XRBs['donor_type'] = donor_type
    df_XRBs['accretor_type'] = accretor_type

    return df_XRBs

Let's test our function to one binary.


In [ ]:
XRB_selection_function(XRB_pop.history[0], XRB_pop.oneline[0], None)

<div class='alert alert-success'>

### Excercise 3

Now let's modify `XRB_selection_function` that we built above, to also include additional information, such as donor and accretor radii, mass loss/transfer rates, orbital period, and eccentricity, as well as the rotational frequency of the donor and the spin of the accreting compact object.

</div>   
 
<div class="alert alert-warning" style="margin-top: 20px">
<details>
    
<b><summary>Hint (click to reveal):</summary></b>

In POSYDON, we keep track of the mass loss/gain rate of each component, as well the mass transfer rate between the two components. The latter is non-zero only for Roche lobe overfilling systems. Let'. save all three quantities, i.e.: `S1_lg_mdot_f`, `S2_lg_mdot_f`, and `lg_mtransfer_rate_f`.

</details>

In [ ]:
# Write here your code for Excercise 3

<div class="alert alert-warning" style="margin-top: 20px">
<details>
    
<b><summary>Solution (click to reveal):</summary></b>

~~~python

import pandas as pd
def XRB_selection_function(history_chunk, oneline_chunk, formation_channels_chunk=None):
    '''A XRB selection function to create a population of XRBs, where we store only the necessary information.'''

    indices = oneline_chunk.index.to_numpy()
    df_XRBs = pd.DataFrame(index = indices)

    #in contrast to the the tutorial for "transient" where we want to be searching the history, here we only care about the final state, so the oneline is sufficient.

    df_XRBs['time'] = oneline_chunk['time_f'] * 1e-6 #Myr
    df_XRBs['time_of_birth'] = oneline_chunk['time_i'] * 1e-6 #Myr
    df_XRBs['metallicity'] = oneline_chunk['metallicity'] #This is not really necessary in our case, since we have one file per metallicity, but it is required by the fanctionality of the "transient population we will use"

    # For the later calculations we are gonna do, it is convenient to switch from an S1 / S2 notation to donnor / accretor notation. 
    # To do that, we will loop over the systems and check which one is the compact object (BH or NS) and assign that to be the accretor, while the other star will be the donor.
    # We will also store the type of compact object in a separate column.
  
    # Compact object types
    compact_types = {'BH', 'NS'}

    # Prepare new columns
    donor_mass = []
    accretor_mass = []
    donor_state = []
    accretor_state = []
    donor_log_R = []
    accretor_log_R = []
    donor_lg_mdot = [] 
    accretor_lg_mdot = []
    donor_surf_avg_omega_div_omega_crit = []
    accretor_spin = []



    # Loop over each system
    for s1, s2, m1, m2, logr1, logr2, mdot1, mdot2, omega1, omega2, spin1, spin2 in zip(oneline_chunk['S1_state_f'], oneline_chunk['S2_state_f'], oneline_chunk['S1_mass_f'], oneline_chunk['S2_mass_f'], oneline_chunk['S1_log_R_f'], oneline_chunk['S2_log_R_f'], oneline_chunk['S1_lg_mdot_f'], oneline_chunk['S2_lg_mdot_f'], oneline_chunk['S1_surf_avg_omega_div_omega_crit_f'], oneline_chunk['S2_surf_avg_omega_div_omega_crit_f'], oneline_chunk['S1_spin_f'], oneline_chunk['S2_spin_f']):
        if s1 in compact_types and s2 not in compact_types:
            accretor_mass.append(m1)
            accretor_state.append(s1)
            accretor_log_R.append(logr1)
            accretor_lg_mdot.append(mdot1)
            accretor_spin.append(spin1)
            donor_mass.append(m2)
            donor_state.append(s2)
            donor_log_R.append(logr2)   
            donor_lg_mdot.append(mdot2)
            donor_surf_avg_omega_div_omega_crit.append(omega2)
        elif s2 in compact_types and s1 not in compact_types:
            accretor_mass.append(m2)
            accretor_state.append(s2)
            accretor_log_R.append(logr2)
            accretor_lg_mdot.append(mdot2)
            accretor_spin.append(spin2)
            donor_mass.append(m1)
            donor_state.append(s1)
            donor_log_R.append(logr1)
            donor_lg_mdot.append(mdot1)
            donor_surf_avg_omega_div_omega_crit.append(omega1)
        else:
            # If neither or both are compact, fill with NaN or original values
            accretor_mass.append(float('nan'))
            accretor_state.append(None)
            accretor_log_R.append(float('nan'))
            accretor_lg_mdot.append(float('nan'))
            accretor_spin.append(float('nan'))
            donor_mass.append(float('nan'))
            donor_state.append(None)
            donor_log_R.append(float('nan'))
            donor_lg_mdot.append(float('nan'))
            donor_surf_avg_omega_div_omega_crit.append(float('nan'))

    # Add new columns to DataFrame
    df_XRBs['donor_mass'] = donor_mass
    df_XRBs['accretor_mass'] = accretor_mass
    df_XRBs['donor_state'] = donor_state
    df_XRBs['accretor_state'] = accretor_state
    df_XRBs['donor_log_R'] = donor_log_R
    df_XRBs['accretor_log_R'] = accretor_log_R
    df_XRBs['donor_lg_mdot'] = donor_lg_mdot
    df_XRBs['accretor_lg_mdot'] = accretor_lg_mdot
    df_XRBs['donor_surf_avg_omega_div_omega_crit'] = donor_surf_avg_omega_div_omega_crit
    df_XRBs['accretor_spin'] = accretor_spin

    # Finaly, add columns that are not associated to teh donor or the accretor
    df_XRBs['orbital_period'] = oneline_chunk['orbital_period_f'].to_numpy()  # days
    df_XRBs['eccentricity'] = oneline_chunk['eccentricity_f'] 
    df_XRBs['lg_mtransfer_rate'] = oneline_chunk['lg_mtransfer_rate_f'] #Msun/yr


    return df_XRBs

~~~
</details>

We just have one more quantity to calculate, bu it is an important one: the X-ray luminosity! Let's try to follow a simplified of the approach describe in Section 2.2. of [Misra et al. (2023)](https://ui.adsabs.harvard.edu/abs/2023A%26A...672A..99M/abstract), where we will neglect for now Be XRBs, as well as geometrical beaming effects of compact objects accreting at super-Eddington rates.

Let's write a function that takes as arguments the masses, radii, and types of the donor and the accretor, as well as the mass transfer rate between the donor and the accretor and the mass-loss rate of the donor star. The latter includes both wind mass loss and mass transfer due to RLO.


In [ ]:
# Constants (cgs)
G = 6.67430e-8
Msun = 1.98847e33
Rsun = 6.957e10
c = 2.99792458e10
year = 3.15576e7

def bh_efficiency_from_spin(a):
    if a is None or not np.isfinite(a):
        return 0.1
    a = float(np.clip(a, 0.0, 0.998))
    return 0.057 + 0.38 * a  # crude approx

def separation_from_period(P_days, Mtot_Msun):
    P = float(P_days) * 86400.0
    Mtot = float(Mtot_Msun) * Msun
    return (G * Mtot * (P / (2.0 * np.pi))**2)**(1.0/3.0)  # cm

def guess_wind_speed(donor_mass_Msun, donor_radius_Rsun, scale=2.6):
    M = float(donor_mass_Msun) * Msun
    R = float(donor_radius_Rsun) * Rsun
    v_esc = np.sqrt(2 * G * M / max(R, 1e-10))
    return scale * v_esc  # cm/s

def bhl_mdot_acc(M_acc_Msun, Mdot_w_Msun_per_yr, a_cm, v_rel_cms, alpha_BHL=1.0):
    Macc = float(M_acc_Msun) * Msun
    Mdot_w = float(max(Mdot_w_Msun_per_yr, 0.0)) * Msun / year  # g/s
    vrel4 = max(v_rel_cms, 1.0)**4
    a2 = max(a_cm, 1.0)**2
    frac = alpha_BHL * (G**2 * Macc**2) / (a2 * vrel4)
    return frac * Mdot_w  # g/s

def _lg_msunyr_to_msunyr(lg_rate):
    if lg_rate is None or not np.isfinite(lg_rate):
        return 0.0
    return 10.0**float(lg_rate)

def compute_Lx_components(
    donor_mass, donor_log_R,
    accretor_mass, accretor_type,
    lg_mtransfer_rate, donor_lg_mdot, accretor_lg_mdot,
    orbital_period_days=None, accretor_spin=None):

    # Efficiency
    if accretor_type == 'NS':
        eta = 0.2
    elif accretor_type == 'BH':
        eta = bh_efficiency_from_spin(accretor_spin)
    else:
        return np.nan, np.nan, np.nan  # non-compact accretor

    # Rates in Msun/yr
    mdot_rlo_Msunyr = _lg_msunyr_to_msunyr(accretor_lg_mdot)          # RLO only
    mdot_wind_only_Msunyr = _lg_msunyr_to_msunyr(donor_lg_mdot) - _lg_msunyr_to_msunyr(lg_mtransfer_rate)

 
    # mdot_rlo_Msunyr = _lg_msunyr_to_msunyr(lg_mtransfer_rate)          # RLO only
    # mdot_donor_total_Msunyr = _lg_msunyr_to_msunyr(donor_lg_mdot)       # winds + possible RLO
    # mdot_wind_only_Msunyr = max(mdot_donor_total_Msunyr - mdot_rlo_Msunyr, 0.0)

    # RLO luminosity (no Edd cap)
    mdot_rlo_gps = mdot_rlo_Msunyr * Msun / year
    Lx_RLO = eta * mdot_rlo_gps * c**2 if mdot_rlo_Msunyr > 0.0 else 0.0

    # BHL luminosity (no Edd cap). If info missing, set to 0.
    Lx_BHL = 0.0

    a = separation_from_period(orbital_period_days, (donor_mass or 0.0) + (accretor_mass or 0.0))
    donor_Rsun = 10.0**donor_log_R
    v_w = guess_wind_speed(donor_mass, donor_Rsun)
    v_orb = 2.0 * np.pi * a / (float(orbital_period_days) * 86400.0)
    v_rel = np.sqrt(v_w**2 + v_orb**2)  # ignore eccentricity
    mdot_bhl_gps = bhl_mdot_acc(accretor_mass, mdot_wind_only_Msunyr, a, v_rel)
    if np.isfinite(mdot_bhl_gps) and mdot_bhl_gps > 0.0:
        Lx_BHL = eta * mdot_bhl_gps * c**2

    Lx_total = Lx_RLO + Lx_BHL
    return float(Lx_RLO), float(Lx_BHL), float(Lx_total)




def XRB_selection_function(history_chunk, oneline_chunk, formation_channels_chunk=None):
    '''A XRB selection function to create a population of XRBs, where we store only the necessary information.'''

    indices = oneline_chunk.index.to_numpy()
    df_XRBs = pd.DataFrame(index = indices)

    #in contrast to the the tutorial for "transient" where we want to be searching the history, here we only care about the final state, so the oneline is sufficient.

    df_XRBs['time'] = oneline_chunk['time_f'] * 1e-6 #Myr
    df_XRBs['time_of_birth'] = oneline_chunk['time_i'] * 1e-6 #Myr
    df_XRBs['metallicity'] = oneline_chunk['metallicity'] #This is not really necessary in our case, since we have one file per metallicity, but it is required by the fanctionality of the "transient population we will use"

    # For the later calculations we are gonna do, it is convenient to switch from an S1 / S2 notation to donnor / accretor notation. 
    # To do that, we will loop over the systems and check which one is the compact object (BH or NS) and assign that to be the accretor, while the other star will be the donor.
    # We will also store the type of compact object in a separate column.
  
    # Compact object types
    compact_types = {'BH', 'NS'}

    # Prepare new columns
    donor_mass = []
    accretor_mass = []
    donor_state = []
    accretor_state = []
    donor_log_R = []
    accretor_log_R = []
    donor_lg_mdot = [] 
    accretor_lg_mdot = []
    donor_surf_avg_omega_div_omega_crit = []
    accretor_spin = []

    Lx_rlo = []
    Lx_bhl = []
    Lx_tot = []

    # Loop over each system
    for s1, s2, m1, m2, logr1, logr2, mdot1, mdot2, omega1, omega2, spin1, spin2 in zip(oneline_chunk['S1_state_f'], oneline_chunk['S2_state_f'], oneline_chunk['S1_mass_f'], oneline_chunk['S2_mass_f'], oneline_chunk['S1_log_R_f'], oneline_chunk['S2_log_R_f'], oneline_chunk['S1_lg_mdot_f'], oneline_chunk['S2_lg_mdot_f'], oneline_chunk['S1_surf_avg_omega_div_omega_crit_f'], oneline_chunk['S2_surf_avg_omega_div_omega_crit_f'], oneline_chunk['S1_spin_f'], oneline_chunk['S2_spin_f']):
        if s1 in compact_types and s2 not in compact_types:
            accretor_mass.append(m1)
            accretor_state.append(s1)
            accretor_log_R.append(logr1)
            accretor_lg_mdot.append(mdot1)
            accretor_spin.append(spin1)
            donor_mass.append(m2)
            donor_state.append(s2)
            donor_log_R.append(logr2)   
            donor_lg_mdot.append(mdot2)
            donor_surf_avg_omega_div_omega_crit.append(omega2)
        elif s2 in compact_types and s1 not in compact_types:
            accretor_mass.append(m2)
            accretor_state.append(s2)
            accretor_log_R.append(logr2)
            accretor_lg_mdot.append(mdot2)
            accretor_spin.append(spin2)
            donor_mass.append(m1)
            donor_state.append(s1)
            donor_log_R.append(logr1)
            donor_lg_mdot.append(mdot1)
            donor_surf_avg_omega_div_omega_crit.append(omega1)
        else:
            # If neither or both are compact, fill with NaN or original values
            accretor_mass.append(float('nan'))
            accretor_state.append(None)
            accretor_log_R.append(float('nan'))
            accretor_lg_mdot.append(float('nan'))
            accretor_spin.append(float('nan'))
            donor_mass.append(float('nan'))
            donor_state.append(None)
            donor_log_R.append(float('nan'))
            donor_lg_mdot.append(float('nan'))
            donor_surf_avg_omega_div_omega_crit.append(float('nan'))

    # Add new columns to DataFrame
    df_XRBs['donor_mass'] = donor_mass
    df_XRBs['accretor_mass'] = accretor_mass
    df_XRBs['donor_state'] = donor_state
    df_XRBs['accretor_state'] = accretor_state
    df_XRBs['donor_log_R'] = donor_log_R
    df_XRBs['accretor_log_R'] = accretor_log_R
    df_XRBs['donor_lg_mdot'] = donor_lg_mdot
    df_XRBs['accretor_lg_mdot'] = accretor_lg_mdot
    df_XRBs['donor_surf_avg_omega_div_omega_crit'] = donor_surf_avg_omega_div_omega_crit
    df_XRBs['accretor_spin'] = accretor_spin

    # Finaly, add columns that are not associated to teh donor or the accretor
    df_XRBs['orbital_period'] = oneline_chunk['orbital_period_f'].to_numpy()  # days
    df_XRBs['eccentricity'] = oneline_chunk['eccentricity_f'] 
    df_XRBs['lg_mtransfer_rate'] = oneline_chunk['lg_mtransfer_rate_f'] #Msun/yr



    for _, r in df_XRBs.iterrows():
        Lx_RLO, Lx_BHL, Lx = compute_Lx_components(
            donor_mass=r.get('donor_mass'),
            donor_log_R=r.get('donor_log_R'),
            accretor_mass=r.get('accretor_mass'),
            accretor_type=r.get('accretor_state'),
            lg_mtransfer_rate=r.get('lg_mtransfer_rate'),    # log10(Msun/yr)
            donor_lg_mdot=r.get('donor_lg_mdot'),            # log10(Msun/yr), may include RLO
            accretor_lg_mdot=r.get('accretor_lg_mdot'),      # log10(Msun/yr), includes only RLO
            orbital_period_days=r.get('orbital_period'),
            accretor_spin=r.get('accretor_spin'))
        Lx_rlo.append(Lx_RLO)
        Lx_bhl.append(Lx_BHL)
        Lx_tot.append(Lx)
    df_XRBs['Lx_rlo'] = Lx_rlo
    df_XRBs['Lx_bhl'] = Lx_bhl
    df_XRBs['Lx_tot'] = Lx_tot

    return df_XRBs


<div class='alert alert-success'>

### Excercise 4
Let's try our new selection function which include the X-ray luminosity function first on just one binary, and then to the entire population.

</div>

In [ ]:
# Write here your code for Excercise 4

<div class="alert alert-warning" style="margin-top: 20px">
<details>
    
<b><summary>Solution (click to reveal):</summary></b>

~~~python

XRB_selection_function(XRB_pop.history[10], XRB_pop.oneline[10], None)

XRBs = XRB_pop.create_transient_population(XRB_selection_function, 'XRB')
~~~

</details>

## 3. X-ray luminosity function (XLF) of extragalactic X-ray binaries  

The **X-ray luminosity function (XLF)** of an extragalactic X-ray binary (XRB) population describes how many XRBs exist in a galaxy as a function of their X-ray luminosity. It can be expressed in two common forms:  

- **Differential XLF**  
  
  $\Phi(L_X) = \frac{dN}{dL_X} \quad \text{or} \quad \frac{dN}{d\log L_X}$

  This gives the number of sources per unit luminosity (or per unit logarithmic luminosity bin).  

- **Cumulative XLF**  
  
  $N(>L_X) = \int_{L_X}^{\infty} \Phi(L)\,dL$ 
  
  This gives the total number of XRBs brighter than a given luminosity \(L_X\).  

---

### Observational usage  

- In practice, **cumulative XLFs** are often shown because:  
  - They avoid the statistical noise introduced by binning sparse data.  
  - Power-law distributions appear as straight lines in log–log space, making slopes easier to measure.  
  - They allow easy comparison between galaxies with different numbers of detected sources.  

---

### Characteristic forms  

- **High-mass XRBs (HMXBs)** in star-forming galaxies:  
  - Cumulative XLF follows a nearly universal power law with slope \(\sim -0.6\).  
  - Normalization scales with the **star-formation rate**.  

- **Low-mass XRBs (LMXBs)** in old populations:  
  - Cumulative XLF shows a **break** at \(L_X \sim 10^{37-38}\,\text{erg/s}\).  
  - Normalization scales with the **stellar mass** of the host galaxy.  

---

👉 When constructing an XLF from simulated POSYDON populations, you can examine either the **differential** or the **cumulative** form, but note that in most extragalactic studies the **cumulative XLF is the standard diagnostic**.

<div class='alert alert-success'>
    
### Excercise 5
Create a a figure with the XLF of our XRB population. Use the total X-ray luminosity for each of the XRBs.

</div>
    
<div class="alert alert-warning" style="margin-top: 20px">
<details>
    
<b><summary>Hint (click to reveal):</summary></b>

It maybe convenient to grab the relevant column from the XRB population object `XRBs.population['Lx_tot']` and convert it to an 1D numpy array, say L, to make the rest of the process easier.

MESA often outputs -99 for for log values of quantities that are zero. We can filter those out, 
by setting a minimum value for the X-ray luminosity that we consider.

The easiest way to compute an XLF is for the X axis to just sort the luminosities array, and for 
the Y axis to create an array of eual lenght to L, filled with ones, then use cumsum and finaly invert invert the order.

</details>

In [ ]:
# Write here your code for Excercise 5

<div class="alert alert-warning" style="margin-top: 20px">
<details>
    
<b><summary>Solution (click to reveal):</summary></b>

~~~python

#Normally, on your own system where LateX is installed, you would load the posydon style like this:
# import matplotlib as mpl
# mpl.style.use(PATH_TO_POSYDON + 'posydon/visualization/posydon.mplstyle')

# But since LateX may not be installed or configured properly on this system, 
# we will ignore the posydon style for now and just prevent LaTeX usage:
import matplotlib as mpl
mpl.rcParams['text.usetex'] = False  # prevent LaTeX usage




L = pd.to_numeric(XRBs.population['Lx_tot']).to_numpy(dtype=float)

L_min = 1e30  # erg/s

# MESA often outputs -99 for for log values of quantities that are zero. We can filter those out, 
# by setting a minimum value for the X-ray luminosity that we consider.
L = L[(L > L_min)]

# The easiest way to compute the cumulative distribution is to sort the luminosities 
# and then use cumsum and invert the order.
L_sorted = np.sort(L)
N_gt = np.cumsum(np.ones_like(L_sorted, dtype=int))[::-1]

fig, axs = plt.subplots(figsize=(5,5))



axs.step(L_sorted, N_gt)
axs.set_xscale('log')
axs.set_yscale('log')
axs.set_xlabel('X-ray luminosity (erg/s)')
axs.set_ylabel('N(>Lx)')
axs.set_xlim(1e35, 1e40)
axs.set_ylim(1, 1e2)
plt.show()


~~~~
<details>

### 3.1  A proper normalisation of our simulated XLF

The Y axis of our XLF is rather arbitrary. Had we run a population of 10 times more binaries, we would have 10 times more XRBs. In order to compare with observations, it is best to normalise our XLF to something physical. For XLFs of star forming galaxies the normalisation of the Y axis is usually per unit of star formation rate $[M_\odot/yr]$. 

In POSYDON, there is a function that gives us the probability of each of the modeled systems in our population per unit of stellar mass. This function function is very flexible allowing us to reweight these probabilities, for distributions of initial binary property distribution, different than the ones we had initially used in our population run. 

Here, we will use it in its simplest form. In our population run, we only modeled systems with primary masses above $5\,\rm M_\odot$, as we did not want to waste cpu time to model binaries that we know would not form XRBs. However, in order to properly calculate the probability of each modeled binary per unit of stellar mass, we would need to extend the IMF of the primary, say down to $0.1\,\rm M_\odot$. For the weights to be calculated properly, one needs to also redifine the limits for the mass ratio distribution: q_min and q_max. Here how this is done:

In [ ]:
# We make a copy of the initial population parameters and change the minimum primary mass to 0.1 Msun
# This is needed for the reweighting to a full IMF down to low masses.
pop_params = XRBs.ini_params.copy()

pop_params['q_min'] = 0.0
pop_params['q_max'] = 1.0
pop_params['primary_mass_min'] = 0.1

weights = XRBs.calculate_model_weights(model_weights_identifier="base_IMF", population_parameters=pop_params)

# These weights have unit of 1/Msun, and give the probability of each modeled system per unit of stellar mass formed in stars.
weights

<div class='alert alert-success'>

### Excercise 6

Let's now properly normalise our XLF, to be per unit of star formation rate. Remember that when we are talking about star formation rate in astronomy, there is always an implicit assumption about the starformation history of the population. The most common, and then one that we assume here is a constant star formation rate over the last 100 Myr. 

Use the weights we calculated above and our assumption of constant star formation rate over the last 100 Myr, to normalise our XLF.

</div>

<div class="alert alert-warning" style="margin-top: 20px">
<details>
    
<b><summary>Hint (click to reveal):</summary></b>

Calculate how much stellar mass is formed over 100 My with a constant star formation of $1 M_\odot / yr$ and multiply the calculated weight with this number. This will make them a proper uniteless weight.

In the previous excercise, for the Y axis, you created an array of equal lenght to L, which you filled with ones. Now you should use the weights array instead.

Don't forget to convert the weights to a numpy array as well.

</details>

In [ ]:
# Write here your code for Excercise 6

<div class="alert alert-warning" style="margin-top: 20px">
<details>
    
<b><summary>Solution (click to reveal):</summary></b>

~~~python

#Normally, on your own system where LateX is installed, you would load the posydon style like this:
# import matplotlib as mpl
# mpl.style.use(PATH_TO_POSYDON + 'posydon/visualization/posydon.mplstyle')

# But since LateX may not be installed or configured properly on this system, 
# we will ignore the posydon style for now and just prevent LaTeX usage:
import matplotlib as mpl
mpl.rcParams['text.usetex'] = False  # prevent LaTeX usage


Total_underlying_mass = 1e8 #Msun  -  1 Msolar/yr * 1e8 yr

L = pd.to_numeric(XRBs.population['Lx_tot']).to_numpy(dtype=float)

L_min = 1e30  # erg/s

# MESA often outputs -99 for for log values of quantities that are zero. We can filter those out, 
# by setting a minimum value for the X-ray luminosity that we consider.
L_masked = L[(L > L_min)]
weights_masked = weights.to_numpy(dtype=float)[(L > L_min)]

# The easiest way to compute the cumulative distribution is to sort the luminosities 
# and then use cumsum and invert the order.
L_sorted = np.sort(L_masked)
weights_sorted = weights_masked[np.argsort(L_masked)]
N_gt = np.cumsum(weights_sorted)[::-1]* Total_underlying_mass

fig, axs = plt.subplots(figsize=(5,5))



axs.step(L_sorted, N_gt)
axs.set_xscale('log')
axs.set_yscale('log')
axs.set_xlabel('X-ray luminosity (erg/s)')
axs.set_ylabel('N(>Lx)/SFR')
axs.set_xlim(1e35, 1e40)
axs.set_ylim(10, 2e3)
plt.show()

~~~

</details>

<div class='alert alert-success'>

### Excercise 7

Now, as a final excercise, let's try to explore a bit further the XLF by spliting it into subpopulations, depending on the type of the accretor (NS or BH), and the type of mass-transfer (RLO or wind-fed).
     
</div>

In [ ]:
# Write here your code for Excercise 7

<div class="alert alert-warning" style="margin-top: 20px">
<details>
    
<b><summary>Solution (click to reveal):</summary></b>

~~~python

Total_underlying_mass = 1e8 #Msun  -  1 Msolar/yr * 1e8 yr


L_min = 1e30  # erg/s
#mask_valid = np.isfinite(Lx_tot) & (Lx_tot > L_min)

# Accretor type masks
is_bh = XRBs.population['accretor_state'] == 'BH'
is_ns = XRBs.population['accretor_state'] == 'NS'

# Accretion mode masks
# RLO if RLO luminosity > BHL luminosity
is_rlo = (XRBs.population['Lx_rlo'] > XRBs.population['Lx_bhl']) & (XRBs.population['Lx_rlo'] > L_min)
is_wind = (XRBs.population['Lx_rlo'] < XRBs.population['Lx_bhl']) & (XRBs.population['Lx_bhl'] > L_min)



def plot_ccdf(vals, w, label):
    x = np.sort(vals)                    # ascending
    y = np.cumsum(w[np.argsort(vals)])[::-1]   # N(>x)
    plt.step(x, y, where='post', label=label)

plt.figure()
plot_ccdf(XRBs.population.loc[is_rlo  & is_bh, 'Lx_tot'].to_numpy(), weights[is_rlo  & is_bh].to_numpy()*Total_underlying_mass, 'RLO + BH')
plot_ccdf(XRBs.population.loc[is_rlo  & is_ns, 'Lx_tot'].to_numpy(), weights[is_rlo  & is_ns].to_numpy()*Total_underlying_mass, 'RLO + NS')
plot_ccdf(XRBs.population.loc[is_wind & is_bh, 'Lx_tot'].to_numpy(), weights[is_wind & is_bh].to_numpy()*Total_underlying_mass, 'Wind + BH')
plot_ccdf(XRBs.population.loc[is_wind & is_ns, 'Lx_tot'].to_numpy(), weights[is_wind & is_ns].to_numpy()*Total_underlying_mass, 'Wind + NS')

plt.xscale('log')
plt.yscale('log')
plt.xlabel('X-ray luminosity (erg/s)')
plt.ylabel('N(>Lx)')
plt.legend()
plt.show()

~~~

<details>

<div class='alert alert-success'>
    
### (Optional) Excercise 8

Compare your simulated XLFs and observed on, e.g. Figure 3 in [Lehmer et al. (2021)](https://arxiv.org/pdf/2011.09476). Do you see any similarities? Do you see any obvious discrepancies? In Figure 3 of [Lehmer et al. (2021)](https://arxiv.org/pdf/2011.09476) you also see the dependence of the XLF to metallicity. You can load a second populations that is available to you at 10% Solar metallicity and see if a similar trend appears in your simulated XLFs. Can you speculate what might be the reason of the dependence of the XLF shape to metallicity?
    
</div>